# Library Usage in Seattle, 2005-2020

## Data Cleaning

The data is the [Checkouts by Title (Physical Items)](https://data.seattle.gov/Community/Checkouts-By-Title-Physical-Items-/5src-czff) dataset from [Seattle Open Data](https://data.seattle.gov/) and was downloaded on December 15, 2020.

This notebook is designed to load a downloaded CSV file, merge it with item-specific information, convert it to a time-series-ready DataFrame, and save that as a compressed pickle file.

*Note: This dataset is updated weekly; the more data, the longer the load times will be.*

In the future, it may be a good idea to look into adding [API](https://dev.socrata.com/foundry/data.seattle.gov/5src-czff) calls into the pipeline, so as to quickly and easily add on the additional weekly data.

*Note: Any cell that uses the built-in magic command* `%%time` *takes a significant (or at least not insignificant) time to run.*

### Import required libraries

In [1]:
# standard dataframe packages
import pandas as pd
import numpy as np

# saving packages
import pickle
import gzip

from functions.data_cleaning import status_update

### Load checkout data

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

Since the data set is so large, I'll specify only the columns that I want in the DataFrame. This will effectively drop the following columns:
- `ID`
- `CheckoutYear`
- `CallNumber`
- `BibNumber`
- `ItemBarcode`
- `ItemType`
    
I want to note that the `ItemType` and `Collection` columns are very similar, but the code in the `Collection` column contains more information within the `category_group` column that I add onto the DataFrame using the `data_dictionary.csv` file ([see below](#Load-other-info-from-data-dictionary-and-merge-onto-checkouts-dataset)). More specifically, the `ItemType` code yields mostly "Miscellaneous" results, whereas the `Collection` code yields differentiates between "Fiction" and "Nonfiction", among others. This could be useful information later on, so I found it best to drop the `ItemType` column.

Note: In the future, I may consider using this [dataset](https://data.seattle.gov/Community/Library-Collection-Inventory/6vkj-f5xf) to add on branch information (i.e. which branch an item was checked out from), although this data is rather limited (due to privacy concerns) and incomplete (only appears to be collected beginning in 2017). In order to do that I would need to use the `BibNumber` column.

#### ⏰ Cell below takes ~14.5 minutes to run. ⏰

In [2]:
%%time

# columns to load
usecols = ['Collection', 'ItemTitle', 'Subjects', 'CheckoutDateTime']

# load data
df = pd.read_csv('data/Checkouts_By_Title__Physical_Items_.csv',
#                  nrows=10000000,
                 usecols=usecols)

# rename columns to my preferred format
df.columns = ['collection', 'title', 'subjects', 'date']

CPU times: user 4min 20s, sys: 4min 39s, total: 9min
Wall time: 14min 1s


*Note: While it only took about 25 seconds to load 10 million rows, it takes about 25 minutes to load 106.5 million rows with the same number (5) of columns. In my latest update, I brought it down to 4 columns, which decreased load time to 15.5 minutes.*

In [3]:
# check shape
df.shape

(106534901, 4)

In [4]:
# take a look
df.head()

,collection,title,subjects,date
0,nadvd,Firewall,"Kidnapping Drama, Video recordings for the hea...",02/13/2008 07:38:00 PM
1,nanf,best baby shower book a complete guide for par...,Showers Parties,07/23/2008 02:53:00 PM
2,nyfic,Uglies,"Fantasy, Teenage girls Fiction, Beauty Persona...",12/23/2009 04:20:00 PM
3,napar,doula guide to birth secrets every pregnant wo...,"Doulas, Childbirth",11/16/2010 12:04:00 PM
4,canf,Salmon a cookbook,Cookery Salmon,04/26/2009 01:29:00 PM


#### ⏰ Cell below takes ~3 minutes to run. ⏰

In [5]:
%%time

# check for nan values
df.isna().sum()

CPU times: user 33.6 s, sys: 1min 9s, total: 1min 43s
Wall time: 3min 19s


collection          0
title          900912
subjects      1649522
date                0
dtype: int64

*NOTE: Even checking for NaN values takes a significant amount of time with this many rows.*

The most important columns (`collection` and `date`) have no NaN values.

In [6]:
# check datatypes
df.dtypes

collection    object
title         object
subjects      object
date          object
dtype: object

### Convert `date` column to datetime

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [7]:
# look at an example before conversion
df.loc[0, 'date']

'02/13/2008 07:38:00 PM'

In [8]:
# specify the format
dt_format = '%m/%d/%Y %I:%M:%S %p'

#### ⏰ Cell below takes ~7 minutes to run. ⏰

In [9]:
%%time

# convert to datetime, dropping the hour-minute-second stamp using the `dt.date` attribute
df['date'] = pd.to_datetime(df.date, format=dt_format).dt.date

# confirm it worked
df.loc[0, 'date']

CPU times: user 6min 27s, sys: 15.6 s, total: 6min 43s
Wall time: 6min 46s


datetime.date(2008, 2, 13)

### Load other info from data dictionary and merge onto checkouts dataset

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [10]:
# load data
dd = pd.read_csv('data/data_dictionary.csv')

# rename columns to my preferred format
dd.columns = ['code', 'description', 'code_type', 'format_group', 'format_subgroup', 
              'category_group', 'category_subgroup', 'age_group']

# take a look
dd.head()

,code,description,code_type,format_group,format_subgroup,category_group,category_subgroup,age_group
0,cazover,CA7-zine collection oversize,ItemCollection,Print,Book,Periodical,NaN,Adult
1,caziner,CA7-zine collection reference,ItemCollection,Print,Book,Periodical,NaN,Adult
2,cazval,CA7-zine collection valuable mat.,ItemCollection,Print,Book,Periodical,NaN,Adult
3,nga,Northgate Branch,Location,NaN,NaN,NaN,NaN,NaN
4,hip,High Point Branch,Location,NaN,NaN,NaN,NaN,NaN


In [11]:
# check shape
dd.shape

(580, 8)

In [12]:
# check datatypes
dd.dtypes

code                 object
description          object
code_type            object
format_group         object
format_subgroup      object
category_group       object
category_subgroup    object
age_group            object
dtype: object

Since I will only be using information from codes whose type is "ItemCollection", I'll subset the data dictionary down to just those rows.

In [13]:
# subset to only collection codes
dd = dd[dd.code_type == 'ItemCollection']

In [14]:
# check for nan values
dd.isna().sum()

code                   0
description            0
code_type              0
format_group           0
format_subgroup       28
category_group         2
category_subgroup    391
age_group              0
dtype: int64

Again, with the size of the eventual DataFrame in mind, I want to drop any unnecessary columns before merging, so I'll drop the following columns:
- `description`, since that is superfluous information for this project
- `code_type`, since that is superfluous information
- `category_subgroup`, since that is mostly NaN values

In [15]:
# drop columns
dd.drop(columns=['description', 'code_type', 'category_subgroup'], inplace=True)

In [16]:
# list of columns to convert
to_convert = ['format_group', 'format_subgroup', 'category_group', 'age_group']

# convert to category datatype
dd[to_convert] = dd[to_convert].apply(pd.Categorical)

In [17]:
# confirm new datatypes
dd.dtypes

code                 object
format_group       category
format_subgroup    category
category_group     category
age_group          category
dtype: object

#### ⏰ Cell below takes ~4 minutes to run. ⏰

In [18]:
%%time

# merge checkouts dataframe with info from data dictionary
df_merged = df.merge(dd, left_on='collection', right_on='code')

# take a look
df_merged.head()

CPU times: user 1min, sys: 1min 50s, total: 2min 51s
Wall time: 4min 23s


,collection,title,subjects,date,code,format_group,format_subgroup,category_group,age_group
0,nadvd,Firewall,"Kidnapping Drama, Video recordings for the hea...",2008-02-13,nadvd,Media,Video Disc,Fiction,Adult
1,nadvd,Marley me,"Comedy films, Married people Drama, Philadelph...",2009-07-03,nadvd,Media,Video Disc,Fiction,Adult
2,nadvd,Six feet under The complete fourth season,"Video recordings for the hearing impaired, Pro...",2008-10-26,nadvd,Media,Video Disc,Fiction,Adult
3,nadvd,Doctor Who The next doctor,"London England Drama, Doctor Who Fictitious ch...",2010-11-10,nadvd,Media,Video Disc,Fiction,Adult
4,nadvd,School ties,"Antisemitism Drama, Video recordings for the h...",2008-12-28,nadvd,Media,Video Disc,Fiction,Adult


### Drop unnecessary columns

I can now drop the `collection` and `code` columns, since those are no longer necessary.

*NOTE: Using the Pandas method `.drop()` was taking well over an hour, so I'm going to try to subset it below, to see if that works any faster.*

#### ⏰ Cell below takes ~38.5 minutes to run. ⏰

In [19]:
%%time

# drop columns
df_merged.drop(columns=['collection', 'code'], inplace=True)

CPU times: user 2min 43s, sys: 18min 6s, total: 20min 50s
Wall time: 44min 42s


### Set `date` column as index

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

I've commented out the below code because now *this* has begun to consistently crash the kernel or zsh shell.

After thinking more about it, I believe I will end up grouping by the date to get raw numbers for each day (not only total checkouts, but total print checkouts and fiction checkouts, etc.), which can be done before setting the `date` column as the index.

In [20]:
# %%time

# # set `date` column as index and sort by index
# df_merged = df_merged.set_index('date').sort_index()

# # take a look
# df_merged.head()

In [21]:
# check shape
df_merged.shape

(106503843, 7)

In [22]:
# check datatypes
df_merged.dtypes

title                object
subjects             object
date                 object
format_group       category
format_subgroup    category
category_group     category
age_group          category
dtype: object

NOTE: I may be able to drop even more columns (thinking especially of `title` and `subjects`), since I'll mostly be looking at sheer numbers of items checked out each day. I'll keep them in for now in case they end up being useful for EDA.

### 💾 Save

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

Due to some several kernel and zsh shell crashes, I'm going to try to save the DataFrame in batches of 10 million rows.

*NOTE: Save time for 10 million rows takes about 5 minutes and the file size is 290MB. Increasing to 20 million rows seemed to increase save time considerably, and so was interrupted before completing.*

In [23]:
# %%time

# # loop through index and multiples of 10 million
# for ind, i in enumerate(range(0, 110000000, 10000000), 1):
    
#     # save (via compressed pickle) a dataframe of 10 million rows, use index for unique file names
#     df_merged.iloc[i:i+10000000].to_pickle(f'data/seattle_lib_{ind}.pkl', compression='gzip')
    
#     # print status/time
#     status_update(f'File {ind} out of 11 saved successfully')

The previous loop appeared to be stuck on the 5th part, so I interrupted the kernel and am attempting to save parts 6-11.

In [24]:
%%time

# loop through index and multiples of 10 million
for ind, i in enumerate(range(50000000, 110000000, 10000000), 5):
    
    # save (via compressed pickle) a dataframe of 10 million rows, use index for unique file names
    df_merged.iloc[i:i+10000000].to_pickle(f'data/seattle_lib_{ind}.pkl', compression='gzip')
    
    # print status/time
    status_update(f'File {ind} out of 11 saved successfully')

Current time = 00:53:04
-----------------------
File 5 out of 11 saved successfully

Current time = 01:18:18
-----------------------
File 6 out of 11 saved successfully

Current time = 02:35:13
-----------------------
File 7 out of 11 saved successfully

Current time = 04:42:51
-----------------------
File 8 out of 11 saved successfully

Current time = 06:06:44
-----------------------
File 9 out of 11 saved successfully

Current time = 06:43:35
-----------------------
File 10 out of 11 saved successfully

CPU times: user 28min 7s, sys: 2h 17min 27s, total: 2h 45min 35s
Wall time: 6h 10min 4s


In [26]:
df_merged.iloc[40000000:50000000].to_pickle(f'data/seattle_lib_4b.pkl', compression='gzip')

In [25]:
please break code

SyntaxError: invalid syntax (<ipython-input-25-b8306b2d38fe>, line 1)

## GRAVEYARD

[[go back to the top](#Library-Usage-in-Seattle,-2005-2020)]

In [ ]:
# %%time

# # list of columns to keep
# keep_cols = ['title', 'subjects', 'date', 'format_group', 'format_subgroup',
#              'category_group', 'age_group']

# # drop columns
# df_merged = df_merged[keep_cols]

In [ ]:
dd[dd.code_type == 'ItemType']

In [ ]:
dd[dd.code == 'nadvd']

In [ ]:
dd[dd.code == 'acdvd']

In [ ]:
dd.code_type.unique()

In [ ]:
dd_item = dd[dd.code_type == 'ItemType'][['code', 'description', 'format_group', 'format_subgroup', 'category_group', 
             'category_subgroup', 'age_group']]

dd_item.head()

In [ ]:
dd_item2 = dd[dd.code_type == 'ItemCollection'][['code', 'description', 'format_group', 'format_subgroup', 'category_group', 
             'category_subgroup', 'age_group']]

dd_item2.head()

In [ ]:
sorted(df.item_type.unique())

In [ ]:
dd_loc = dd[dd.code_type == 'Location'][['code', 'description']]

dd_loc.head()

In [ ]:
test = df.merge(dd_item, left_on='item_type', right_on='code')
# test = test.merge(dd_loc, left_on='collection', right_on='code')

test.head()

In [ ]:
test.isna().sum()

In [ ]:
test.format_group.value_counts()

In [ ]:
test.collection.unique()

In [ ]:
test.shape

In [ ]:
test2 = df.merge(dd_item2, left_on='Collection', right_on='code')

test2.head()

In [ ]:
test.groupby('format_group').category_group.value_counts()

In [ ]:
test2.groupby('format_group').category_group.value_counts()

In [ ]:
dd[dd.code == 'nybot']

In [ ]:
sorted(df.collection.unique())

In [ ]:
dd_loc.code.unique()

In [ ]:
[cod for cod in df.collection.unique() if cod in dd_loc.code.unique()]

#### NOTE: Using the code below on 10 million rows is almost 39% faster and results in a saved file that is *nearly the same size as the original CSV file* (file is 23.22GB!).

In [ ]:
# %%time

# df_merged.to_hdf('data/seattle_lib_temp_ten_mil__alt.hdf', 'mydata', format='table', mode='w')

In [ ]:
df.title.value_counts().head(10)

In [ ]:
df[df.title=='reader'].head()